# Generate Nice Instagram Captions using Gemini 2.0 Flash

In this tutorial, I’ll show you how to leverage the Gemini 2.0 Flash model (via the Generative AI API) to effortlessly generate multiple caption options for your Instagram posts.

I know that probably there is an agent out there that does it. The intention here is to focus on those people who likes to someday CODE de agents and not just use them.

I am a data scientist specializing in NLP, so my target here is to show how to use the API to build more personalized code, learning to tweak it to be just the way YOU want.

---

👉 This is a very simple tutorial, just to show the basics on how to set up the Google's API and use Gemini 2.0 Flash — and any other Google's model available — so you can then build your own projects.

In [ ]:
!pip install -U langgraph anthropic langchain_anthropic --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 154.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 10.8 MB/s eta 0:00:00


In [ ]:
import google.generativeai as genai

from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [ ]:
# Replace with your actual Google API key
GOOGLE_API_KEY = "YOUR_GOOGLE_API_KEY" # <<< --- REPLACE WITH YOUR KEY ---
genai.configure(api_key=GOOGLE_API_KEY)


In [ ]:
# Define a state schema that holds a list of messages.
class State(TypedDict):
  messages: Annotated[list, add_messages]

In [ ]:
# Initialize a LangGraph instance with the state.
graph_builder = StateGraph(State)

# # --- LLM Initialization ---
# Use a readily available model like gemini-1.5-flash or gemini-pro
# Note: "gemini-2.0-flash" might not be available via the standard API yet.
# Check Google AI documentation for the latest model names.

try:
    model = genai.GenerativeModel("gemini-2.0-flash")
except Exception as e:
    print(f"Warning: Could not initialize gemini-1.5-flash. Trying gemini-pro. Error: {e}")
    try:
        # Fallback option
        model = genai.GenerativeModel("gemini-pro")
    except Exception as final_e:
        print(f"Error: Could not initialize any Gemini model. Please check your API key and model availability. Error: {final_e}")
        exit() # Exit if no model can be loaded


# ------------------------- NODE FUNCTION -------------------------
# Define the node function that generates post ideas
def generate_post_ideas(state: State):
    """Generates LinkedIn post ideas based on the conversation history."""
    print(f"\n--- Entering generate_post_ideas ---")
    print(f"Received state messages type: {type(state['messages'])}")
    print(f"Received state messages content: {state['messages']}")

    # Convert LangChain message objects to the format Google API expects
    # Google API expects a list of {'role': 'user'/'model', 'parts': [text]}
    current_messages = state["messages"]
    history_for_google = []
    for msg in current_messages:
        role = 'user' if isinstance(msg, HumanMessage) else 'model'
        # Ensure content is treated as text
        content_text = str(msg.content)
        history_for_google.append({'role': role, 'parts': [content_text]})

    print(f"Formatted history for Google API: {history_for_google}")

    # Define the number of post ideas
    num_ideas = 5

    # Construct a structured prompt for LinkedIn post ideas.
    prompt_content = f"""Generate {num_ideas} creative and actionable LinkedIn post ideas focused on professional networking, considering the previous conversation context if any.
                         Ensure each idea is concise and suitable for a LinkedIn post. Present the ideas in an enumerated format as follows:

                         Suggestion 1: [LinkedIn Post Idea 1]
                         Suggestion 2: [LinkedIn Post Idea 2]
                         Suggestion 3: [LinkedIn Post Idea 3]
                         Suggestion 4: [LinkedIn Post Idea 4]
                         Suggestion 5: [LinkedIn Post Idea 5]
                         """

    # The prompt itself is the user message for the API call
    prompt_for_google = [{'role': 'user', 'parts': [prompt_content]}]

    try:
        # Use generate_content for stateless calls or if chat history management is simple
        # Combine history and the new prompt
        full_prompt_for_google = history_for_google + prompt_for_google

        # Make the API call
        response = model.generate_content(full_prompt_for_google)

        # Debug: Print raw response
        print(f"Raw Google API response: {response}")

        # Extract text, handling potential errors or empty responses
        if response.parts:
            generated_text = response.text
        else:
            # Handle cases where the response might be blocked or empty
            generated_text = "Model did not generate a response. Check safety settings or prompt."
            if hasattr(response, 'prompt_feedback'):
                 generated_text += f" (Feedback: {response.prompt_feedback})"


    except Exception as e:
        print(f"Error during Google API call: {e}")
        # Handle API errors gracefully
        generated_text = f"An error occurred while generating ideas: {e}"


    print(f"Generated text: {generated_text}")
    print(f"--- Exiting generate_post_ideas ---")

    # Return the response as an AIMessage for LangGraph state
    # Crucially, ensure the returned message is added back correctly by add_messages
    return {"messages": [AIMessage(content=generated_text)]}



In [ ]:
# ------------------------- GRAPH CONSTRUCTION ------------------------- #

# Adding the node to the graph
graph_builder.add_node("post_idea_generator", generate_post_ideas)

# Define the graph flow
graph_builder.add_edge(START, "post_idea_generator")
graph_builder.add_edge("post_idea_generator", END)

# Compile the graph into a runnable object
graph = graph_builder.compile()


In [ ]:
# ------------------------- RUN THE AGENT ------------------------- #

# --- Agent Runner ---
def run_agent():
    print("LinkedIn Post Ideas Generator. Type 'quit' to exit.")
    while True:
        user_input = input("Enter some context or leave blank: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Exiting agent. Goodbye!")
            break

        if not user_input.strip():
            print("No context provided, generating general networking ideas.")
            # Provide a minimal input if none is given, so the message list isn't empty
            initial_message_content = "General professional networking"
        else:
            initial_message_content = user_input

        # Initialize the state correctly for add_messages
        # It expects a list containing message objects or dicts it can convert
        initial_state = {"messages": [HumanMessage(content=initial_message_content)]}
        # Alternative if the above causes issues (less likely with add_messages):
        # initial_state = {"messages": [{"role": "user", "content": initial_message_content}]}


        print(f"\n--- Running Graph ---")
        print(f"Initial State: {initial_state}")

        # Run the graph and stream the output
        try:
            for event in graph.stream(initial_state):
                # Print the full event for debugging if needed
                # print(f"\nGraph Event: {event}")
                for node_name, node_output in event.items():
                    # Check if the output contains messages
                    if isinstance(node_output, dict) and "messages" in node_output:
                         # Get the last message added by the node
                        last_message = node_output["messages"][-1]
                        # Ensure content is treated as string before printing
                        print(f"\nAgent ({node_name}):\n{str(last_message.content)}")
                    else:
                        # Handle cases where the event might not be message output
                        # print(f"Event from {node_name}: {node_output}")
                        pass # Ignore non-message events for now

        except Exception as e:
            print(f"\n--- Error during graph execution ---")
            print(f"An error occurred: {e}")
            import traceback
            traceback.print_exc() # Print detailed traceback for debugging

        print(f"--- Graph Run Finished ---")

In [ ]:
 run_agent()

LinkedIn Post Ideas Generator. Type 'quit' to exit.
Enter some context or leave blank: NATURAL LANGUAGE PROCESSING AND AI

--- Running Graph ---
Initial State: {'messages': [HumanMessage(content='NATURAL LANGUAGE PROCESSING AND AI', additional_kwargs={}, response_metadata={})]}

--- Entering generate_post_ideas ---
Received state messages type: <class 'list'>
Received state messages content: [HumanMessage(content='NATURAL LANGUAGE PROCESSING AND AI', additional_kwargs={}, response_metadata={}, id='8cee91bc-2b9e-46af-b427-789fefee7ee3')]
Formatted history for Google API: [{'role': 'user', 'parts': ['NATURAL LANGUAGE PROCESSING AND AI']}]
Raw Google API response: response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "Suggestion 1: **Just finished an incredible NLP project leveraging AI to [mention specific applica